# Размещаем бота на HEROKU #

Способов существует очень много. Здесь описан тот, который использовал я.

Для начала нам необходимо установить ```heroku``` и ```git``` на ваш компьютер. Скачать можно по следующим ссылкам:

https://devcenter.heroku.com/articles/heroku-cli

https://git-scm.com/downloads

Регистрируемся на https://signup.heroku.com/

<img src='pic/1.png' width='400'>

На почту приходит письмо с ссылкой для подтвержления. Переходим по ссылке и устанавливаем пароль для нашего аккаунта.

Теперь можно переходить на https://dashboard.heroku.com/apps

Теперь нам необходимо создать ```application```. Для этого нажимаем на ```create new app```

<img src='pic/2.png' width='200'>

Вводим название нашего приложения (придумываем сами). Выбираем регион (я выбираю Europe)

<img src='pic/3.png' width="400">

Теперь собственно настало время создавать нашего бота. В качестве примера, я напишу бота, который на любое ваше сообщение будет отвечать "Привет, я бот".

Существует два способа, как ваш бот может работать с ```Telegram```

1. ```Long polling```. Это когда бот постоянно обращается на сервер телеграмма и спрашиват "Телеграмм, есть ли для меня сообщения, или нет". Если есть то выполняется код по обработке этих сообщений.

2. ```Web Hook```. Здесь бот просто сидит и ждет, когда же ему телеграмм отправит сообщение.

Напишем бота сначала на ```Long polling```, а затем перепишем его с использованием ```Web Hook```

Для того, что бы бот успешно работал на ```heroku```, необходимо создать следующие файлы в директории с нашим проектом:

**Первое** - файл ```requirements.txt```. Это обычный текстовый файл, в котором перечислены библиотеки (с указанием версии), которые нам потребуются. Heroku при установке нашего скрипта, установит эти библиотеки.

Содержимое этого файла в нашем случае выглядит следующим образом:


```
pyTelegramBotAPI==3.7.3
python-dotenv==0.15.0
```

Версии устанавливаемых библиотек можно посмотреть в консоли (cmd) с помощью команды ```pip freeze```

<img src='pic/4.png' width='300'>

**Второе** - файл ```Procfile```. Этот файл не должен иметь никакого расширения (например .txt). Здесь мы указываем следующее заклинание, которое говорит **Heroku, запусти скрипт main.py на python3**:

```worker: python3 main.py```

, где main.py - имя нашего скрипта

Для того, что бы создать этот ```Procfile```, создаем в нашей директории с проектом бота обычный текстовый файлик. Называем его к примеру ```somehow.txt```. Пишем туда наше заклинание.

<img src='pic/5.png' width='500'>

Теперь через командную строку переименовываем его из ```somehow.txt``` в ```Procfile```:

* заходим в директорию, где лежит наш файлик ```somehow.txt``` через команду ```cd [путь]``` (у меня эта команда выглядит следующим образом ```cd C:\Users\alpex\HSE_study\Git\Simple_echo_bot```)

* вводим команду ```ren somehow.txt Procfile``` - переименовываем из *somehow.txt* в *Procfile*

и вуаля

<img src='pic/6.png' width='500'>

**Третье** - теперь нам потребуется файлик ```.env```

Обьясню зачем

Вы своего бота будете заливать на Github, и что бы не держать TOKEN от вашего бота в коде, в Heroku есть возможно записать его в переменные окружения. По сути дела, это и есть файлик .env, в котором содержатся пара ключ - значение.

На самом Heroku сделать это очень просто. На страничке вашего приложения на Heroku заходим в ```Sittings```, выбираем ```reveal config vars``` и в поле **key** вводим TOKEN, а в поле **value** собственно вводим значение нашего токена, который дал нам телеграмм.

<img src='pic/7.png' width='800'>

Нажимаем кнопку ```add```

Для того, что бы тестировать бота на нашем компьютере, нам тоже нужно создать аналогичный файл ```.env```.

Создаем текстовый файл, например ```somehow.txt```, записываем туда наши переменные окружения (в нашем случае это TOKEN) в формате

```
KEY=VALUE
```

<img src='pic/8.png' width='500'>

Переименовываем файл ```somehow.txt``` в ```.env``` точно также, как мы делали для ```Procfile```:

вводим в командной строке ```ren somehow.txt .env```

**ВАЖНО! перед тем, как заливать все файлы с ботом на GitHub, убедитесь что файл .env прописан в .gitignore**. Иначе ваш TOKEN будет видет всему интернету.

Собственно код, который содержится в ```main.py```

```python
import telebot
from os import environ
from dotenv import load_dotenv

# загружаем переменные окружения из файла .env
load_dotenv()
TOKEN = environ.get('TOKEN')

# создаем обьект bot
bot = telebot.TeleBot(token=TOKEN)


# обработчик сообщений, который на любое текстовое сообщение от пользователя
# отвечает 'Привет, я бот'
@bot.message_handler(content_types=["text"])
def say_hi(message):
    bot.send_message(message.chat.id, 'Привет, я бот')


# main функция с polling
# наш бот постоянно бегает на сервер Telegram с вопросом "Для меня есть сообщения?"
def main():
    bot.polling(none_stop=True)


if __name__ == '__main__':
    main()

```

Теперь зальём нашего бота на GitHub:

* В командной строке в папке с нашим ботом вводим ```git init```
* В GitHub Desktop выбираем File -> Add local repository..
* Выбираем папку с нашим ботом
* Push-аем на GitHub

Файлы бота появились на GitHub

Теперь репозиторий на GitHub с ботом можно связать с Heroku app. Для этого:
* Заходим на Heroku в наш созданный ранее application
* Заходим во вкладку Deploy
* В Deployment method выбираем GitHub
* Connect to GitHub

<img src='pic/9.png' width='800'>

Чуть ниже включаем authomatic deploy. Теперь при обновлении файлов на GitHub, наш бот на Heroku также будет автоматически обновляться

<img src='pic/10.png' width='800'>

И еще чуть ниже запускаем первый Deploy нашего бота

<img src='pic/11.png' width='800'>

Должно появиться сообщение ```Your app was successfully deployed.```. Это значит, что наше приложение загружено на Heroku и готово к запуску.

Запускаем приложение:

* Заходим во вкладку ```Resources```
* Нажимаем на кнопку редактирования
* Нажимаем на ползунок (что бы включить приложение, он должен оказаться справа)
* Нажимаем ```confirm```


<img src='pic/12.png' width='800'>

<img src='pic/13.png' width='800'>

Все, теперь бот работает на сервере Heroku

### Настраиваем web hook ###

Делать это будем с помощью библиотеки ```Flask```

Добавляем ее в файлик ```requirements.txt```:

```
pyTelegramBotAPI==3.7.3
python-dotenv==0.15.0
Flask==1.1.2
```

В ```Procfile``` заменяем ```worker``` на ```web``` ибо согласно документации:

```
A Heroku app’s web process type is special: it’s the only process type that can receive external HTTP traffic from Heroku’s routers. If your app includes a web server, you should declare it as your app’s web process.
```

Теперь наше заклинание в Process файле выглядит следующим образом

```web: python3 main.py```

Меняем на код в ```main.py```

```python
import telebot
from os import environ
from dotenv import load_dotenv
from flask import Flask, request

# загружаем переменные окружения из файла .env
load_dotenv()
TOKEN = environ.get('TOKEN')
PORT = int(environ.get('PORT', 5000))  # второе значение по умолчанию, если не будет найдено первое

# создаем обьект bot и обьект server
bot = telebot.TeleBot(token=TOKEN)
server = Flask(__name__)


# обработчик сообщений, который на любое текстовое сообщение от пользователя
# отвечает 'Привет, я бот'
@bot.message_handler(content_types=["text"])
def say_hi(message):
    bot.send_message(message.chat.id, 'Привет, я бот')


@server.route('/' + TOKEN, methods=['POST'])
def getMessage():
    bot.process_new_updates([telebot.types.Update.de_json(request.stream.read().decode("utf-8"))])
    return "!", 200


@server.route("/")
def webhook():
    bot.remove_webhook()
    bot.set_webhook(url='https://test-app-hse.herokuapp.com/' + TOKEN)
    return "!", 200
# Обратите внимание, что значение url у вас будет своим. Обычно это в формате
# https:// [имя вашего app на heroku] .herokuapp.com/ 


# main функция с Web Hook
# наш бот смиренно ждет сообщение от Telegram
def main():
    server.run(host="0.0.0.0", port=PORT)


if __name__ == '__main__':
    main()
```

В переменные окружения на Heroku добавляем переменную ```PORT``` (см. выше как я это делал для ```TOKEN```)

Значения переменной ```PORT``` выбираем из списка портов: **443**, **80**, **88**, **8443**. Ибо согласно документации TelegramAPI:

```Ports currently supported for Webhooks: 443, 80, 88, 8443```

Заливаем все это на GitHub. Heroku должен обновить app автоматически.

Запускаем ```web``` также как я делал для ```worker```